# Uniswap Analysis

## Setup

In [4]:
import requests
import pandas as pd
import time

## Arbitrum

### Macro

In [2]:
endpoint_url = 'https://api.thegraph.com/subgraphs/name/ianlapham/arbitrum-minimal'
payload = {"operationName": "uniswapDayDatas",
          "query": "query uniswapDayDatas($startTime: Int!, $skip: Int!) {\n  uniswapDayDatas(\n    first: 1000\n    skip: $skip\n    subgraphError: allow\n    where: {date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"startTime": 1620423606, 
                        "skip": 0}
          }
respose = requests.post(endpoint_url, json=payload).json()

In [3]:
days = respose['data']['uniswapDayDatas']
arb_df = pd.DataFrame(days)
arb_df.set_index('date', inplace=True)
arb_df.index = pd.to_datetime(arb_df.index, unit='s', origin='unix')
#arb_df.to_csv('arb')
arb_df.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-06-01,0,0,0
2021-06-02,0,0,0
2021-06-04,0,0,0
2021-06-17,0,0,0
2021-06-23,0,0,0


### pools

In [5]:
from messari.blockexplorers import Arbiscan

In [6]:
arbiscan = Arbiscan()
uniswap_factory = '0x1F98431c8aD98523631AE4a59f267346ea31F984'
pool_created_keccak = '0x783cca1c0412dd0d695e784568c96da2e9c22ff989357a2e8b1d9b2b4e6b7118'
logs = arbiscan.get_logs(uniswap_factory, 0, to_block='latest', topic0=pool_created_keccak)

In [7]:
hex_timestamps = logs['timeStamp'].tolist()
unix_timestamps = [int(timestamp, 16) for timestamp in hex_timestamps]

count = 0
dict_list = []
for timestamp in unix_timestamps:
    count += 1
    time_dict = {'date': timestamp, 'totalPools': count}
    dict_list.append(time_dict)
    
pools_df = pd.DataFrame(dict_list)
pools_df.set_index('date', inplace=True)
pools_df.index = pd.to_datetime(pools_df.index, unit='s', origin='unix')
#pools_df.to_csv('arbitrum_pools')
pools_df.head()

,totalPools
date,
2021-06-01 21:45:38,1
2021-06-04 09:53:27,2
2021-06-04 17:07:11,3
2021-06-17 17:19:07,4
2021-06-23 22:02:53,5


### Micro

In [4]:
# ETH/USDC .05%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443",
                        "startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
eth_usdc_05 = pd.DataFrame(days)
eth_usdc_05.set_index('date', inplace=True)
eth_usdc_05.index = pd.to_datetime(eth_usdc_05.index, unit='s', origin='unix')
#eth_usdc_05.to_csv('eth_usdc_arb_05')
eth_usdc_05.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-07-12,0,0,0
2021-08-31,0,0,0
2021-09-01,191.2432958845499654671343011317985,167467.552514278344355507459950432,382486.5917690999309342686022635969
2021-09-02,101.7251195460422009437940841144394,225968.1197924498605921010077025047,203450.2390920844018875881682288793
2021-09-03,295.5927549073871837123587542257189,231279.9137152784843522220631601813,591185.5098147743674247175084514368


In [5]:
# ETH/USDC .3%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x17c14D2c404D167802b16C450d3c99F88F2c4F4d".lower(),
                        "startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
eth_usdc_3 = pd.DataFrame(days)
eth_usdc_3.set_index('date', inplace=True)
eth_usdc_3.index = pd.to_datetime(eth_usdc_3.index, unit='s', origin='unix')
#eth_usdc_3.to_csv('eth_usdc_arb_3')
eth_usdc_3.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-06-17,0,0,0
2021-07-12,0,0,0
2021-07-14,0,0,0
2021-08-27,0,0,0
2021-08-29,0,0,0


In [6]:
# WBTC/ETH .3%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x149e36e72726e0bcea5c59d40df2c43f60f5a22d",
                        "startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
wbtc_eth_3 = pd.DataFrame(days)
wbtc_eth_3.set_index('date', inplace=True)
wbtc_eth_3.index = pd.to_datetime(wbtc_eth_3.index, unit='s', origin='unix')
#wbtc_eth_3.to_csv('wbtc_eth_arb_3')
wbtc_eth_3.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-06-23,0,0,0
2021-09-01,0.004773251500755302690566482180886203,34.95359163308263073980906950297379,1.591083833585100896855494060295401
2021-09-02,0,36.74964423457125530784696197228787,0
2021-09-03,0.2672768903835215100247087873531662,173.774740109652131243632604836759,89.09229679450717000823626245105539
2021-09-08,79.97415913743949601820879950601567,9860.571281278631197809712931347506,26658.05304581316533940293316867188


In [7]:
# WBTC/ETH .05%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x2f5e87C9312fa29aed5c179E456625D79015299c".lower(),
                        "startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
wbtc_eth_05 = pd.DataFrame(days)
wbtc_eth_05.set_index('date', inplace=True)
wbtc_eth_05.index = pd.to_datetime(wbtc_eth_05.index, unit='s', origin='unix')
#wbtc_eth_05.to_csv('wbtc_eth_arb_05')
wbtc_eth_05.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-09-12,0.01202444837624715126768108536914834,262.4855083677611804422296649586496,24.0488967524943025353621707382967
2021-09-13,15.88162732613658664112911131300303,4905.419591330594073915014249846976,31763.25465227317328225822262600614
2021-09-14,7.490490225651849646626797039113466,13665.67109272399461321223777042,14980.98045130369929325359407822693
2021-09-15,37.23797365397710806906539545915166,6171.756508336434683320537622021533,74475.94730795421613813079091830339
2021-09-16,14.95347793864416549056687381674847,6154.260428220815694376023216047711,29906.95587728833098113374763349689


In [8]:
# usdt/usdc .05%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x13398E27a21Be1218b6900cbEDF677571df42A48".lower(),
                        "startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
usdc_usdt_05 = pd.DataFrame(days)
usdc_usdt_05.set_index('date', inplace=True)
usdc_usdt_05.index = pd.to_datetime(usdc_usdt_05.index, unit='s', origin='unix')
#usdc_usdt_05.to_csv('usdc_usdt_arb_05')
usdc_usdt_05.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-09-01,0,1202.679054,0
2021-09-02,0,806.4015460000000000000000000000002,0
2021-09-03,0,1042.77883,0
2021-09-04,0,5085.890261000000000000000000000001,0
2021-09-05,0,4058.996433000000000000000000000003,0


## Polygon

### Macro

In [9]:
endpoint_url = 'https://api.thegraph.com/subgraphs/name/ianlapham/uniswap-v3-polygon'
payload = {"operationName": "uniswapDayDatas",
          "query": "query uniswapDayDatas($startTime: Int!, $skip: Int!) {\n  uniswapDayDatas(\n    first: 1000\n    skip: $skip\n    subgraphError: allow\n    where: {date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()

In [10]:
days = response['data']['uniswapDayDatas']
poly_df = pd.DataFrame(days)
poly_df.set_index('date', inplace=True)
poly_df.index = pd.to_datetime(poly_df.index, unit='s', origin='unix')
#poly_df.to_csv('poly')
poly_df.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-12-20,0,1639.153736352038603012062640734921,0
2021-12-21,19.83350189993129015154331509481177,181061.7686362693174597590782642971,6931.678345450813748534796878655774
2021-12-22,989.9243629737167066723920167060548,4251102.683971779902455841400337496,412332.1093658563375270368994241911
2021-12-23,18142.93104552857879905863384350847,9939534.21699501552903134789041357,18165613.36073950156755948560628875
2021-12-24,24735.90330788746117723738067532567,13708150.83721392801465737781175082,30211866.54281128706477692557296161


### Pools

In [8]:
from messari.blockexplorers import Polygonscan

In [9]:
polyscan = Polygonscan()
uniswap_factory = '0x1F98431c8aD98523631AE4a59f267346ea31F984'
pool_created_keccak = '0x783cca1c0412dd0d695e784568c96da2e9c22ff989357a2e8b1d9b2b4e6b7118'
logs = polyscan.get_logs(uniswap_factory, 0, to_block='latest', topic0=pool_created_keccak)

In [10]:
hex_timestamps = logs['timeStamp'].tolist()
unix_timestamps = [int(timestamp, 16) for timestamp in hex_timestamps]

count = 0
dict_list = []
for timestamp in unix_timestamps:
    count += 1
    time_dict = {'date': timestamp, 'totalPools': count}
    dict_list.append(time_dict)
    
pools_df = pd.DataFrame(dict_list)
pools_df.set_index('date', inplace=True)
pools_df.index = pd.to_datetime(pools_df.index, unit='s', origin='unix')
#pools_df.to_csv('polygon_pools')
pools_df.head()

,totalPools
date,
2021-12-20 17:54:32,1
2021-12-20 20:28:00,2
2021-12-20 22:35:38,3
2021-12-21 01:54:21,4
2021-12-21 17:55:04,5


### Micro

In [24]:
# USDC/WETH .05%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x45dDa9cb7c25131DF268515131f647d726f50608",
                        "startTime": 1619170975,
                        "skip": 0}
          }
respose = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
usdc_weth_05 = pd.DataFrame(days)
usdc_weth_05.set_index('date', inplace=True)
usdc_weth_05.index = pd.to_datetime(usdc_weth_05.index, unit='s', origin='unix')
#usdc_weth_05.to_csv('eth_usdc_poly_05')
usdc_weth_05.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-12-22,26.351909048,228229.2910501979984413841816180264,52703.818096
2021-12-23,249.457532391,890239.4515665691203404443528540822,498915.064782
2021-12-24,396.421572525,1081621.507755543567231942651012664,792843.14505
2021-12-25,270.6255200335,1098410.246194091712810678790414872,541251.040067
2021-12-26,486.702256215,10598960.85200910613735991093269903,973404.51243


In [23]:
# USDC/WETH .3%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x0e44cEb592AcFC5D3F09D996302eB4C499ff8c10",
                        "startTime": 1619170975,
                        "skip": 0}
          }
respose = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
usdc_weth_3 = pd.DataFrame(days)
usdc_weth_3.set_index('date', inplace=True)
usdc_weth_3.index = pd.to_datetime(usdc_weth_3.index, unit='s', origin='unix')
#usdc_weth_3.to_csv('eth_usdc_poly_3')
usdc_weth_3.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-12-22,26.351909048,228229.2910501979984413841816180264,52703.818096
2021-12-23,249.457532391,890239.4515665691203404443528540822,498915.064782
2021-12-24,396.421572525,1081621.507755543567231942651012664,792843.14505
2021-12-25,270.6255200335,1098410.246194091712810678790414872,541251.040067
2021-12-26,486.702256215,10598960.85200910613735991093269903,973404.51243


In [22]:
# WBTC/WETH .05%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x50eaEDB835021E4A108B7290636d62E9765cc6d7",
                        "startTime": 1619170975,
                        "skip": 0}
          }
respose = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
wbtc_weth_05 = pd.DataFrame(days)
wbtc_weth_05.set_index('date', inplace=True)
wbtc_weth_05.index = pd.to_datetime(wbtc_weth_05.index, unit='s', origin='unix')
#wbtc_weth_05.to_csv('wbtc_weth_poly_05')
wbtc_weth_05.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-12-22,26.351909048,228229.2910501979984413841816180264,52703.818096
2021-12-23,249.457532391,890239.4515665691203404443528540822,498915.064782
2021-12-24,396.421572525,1081621.507755543567231942651012664,792843.14505
2021-12-25,270.6255200335,1098410.246194091712810678790414872,541251.040067
2021-12-26,486.702256215,10598960.85200910613735991093269903,973404.51243


In [21]:
# WBTC/WETH .3%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0xfe343675878100b344802A6763fd373fDeed07A4",
                        "startTime": 1619170975,
                        "skip": 0}
          }
respose = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
wbtc_weth_3 = pd.DataFrame(days)
wbtc_weth_3.set_index('date', inplace=True)
wbtc_weth_3.index = pd.to_datetime(wbtc_weth_3.index, unit='s', origin='unix')
#wbtc_weth_3.to_csv('wbtc_weth_poly_3')
wbtc_weth_3.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-12-22,26.351909048,228229.2910501979984413841816180264,52703.818096
2021-12-23,249.457532391,890239.4515665691203404443528540822,498915.064782
2021-12-24,396.421572525,1081621.507755543567231942651012664,792843.14505
2021-12-25,270.6255200335,1098410.246194091712810678790414872,541251.040067
2021-12-26,486.702256215,10598960.85200910613735991093269903,973404.51243


In [20]:
# USDC/USDT .05%
payload = {"operationName": "poolDayDatas",
          "query": "query poolDayDatas($startTime: Int!, $skip: Int!, $address: Bytes!) {\n  poolDayDatas(\n    first: 1000\n    skip: $skip\n    where: {pool: $address, date_gt: $startTime}\n    orderBy: date\n    orderDirection: asc\n    subgraphError: allow\n  ) {\n    date\n    volumeUSD\n    tvlUSD\n    feesUSD\n  }\n}\n",
          "variables": {"address": "0x3f5228d0e7d75467366be7de2c31d0d098ba2c23",
                        "startTime": 1619170975,
                        "skip": 0}
          }
response = requests.post(endpoint_url, json=payload).json()
days = response['data']['poolDayDatas']
usdc_usdt_05 = pd.DataFrame(days)
usdc_usdt_05.set_index('date', inplace=True)
usdc_usdt_05.index = pd.to_datetime(usdc_usdt_05.index, unit='s', origin='unix')
#usdc_usdt_05.to_csv('usdc_usdt_poly_05')
usdc_usdt_05.head()

,feesUSD,tvlUSD,volumeUSD
date,,,
2021-12-22,26.351909048,228229.2910501979984413841816180264,52703.818096
2021-12-23,249.457532391,890239.4515665691203404443528540822,498915.064782
2021-12-24,396.421572525,1081621.507755543567231942651012664,792843.14505
2021-12-25,270.6255200335,1098410.246194091712810678790414872,541251.040067
2021-12-26,486.702256215,10598960.85200910613735991093269903,973404.51243
